### Number of wet-dry and wet-wet days per month

#### Code developed to calculate the number of wet days following a dry days, and the number 
of wet days following a wet day per month using a time series of precipitation data

Created on Wed Apr 13 11:41:24 2022 

Thiago Victor Medeiros do Nascimento

In [1]:
import numpy as np
import os
import pandas as pd
import datetime

In [2]:
os.chdir(r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\swat\p-wetwet-drydry') #Set the directory

# Read the data
path =r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\swat\p-wetwet-drydry\meteo-data.xlsx'
meteodata=pd.read_excel(path)
meteodata

,Date,Tmed (ºC),Tmax (ºC),Tmin (ºC),HRmed (%),HRmax (%),HRmin (%),RSG (kj/m2),DV (graus),VVmed (m/s),VVmax (m/s),P (mm),Tmed Relva(ºC),Tmax Relva(ºC),Tmin Relva(ºC),ET0 (mm)
0,2001-09-01 00:00:00,23.50,35.80,13.10,53.60,93.40,20.40,26200.00,225.00,1.60,6.10,0.0,28.60,47.60,16.20,6.00
1,2001-09-02 00:00:00,25.10,33.70,16.70,48.50,69.80,27.70,25483.00,277.00,2.20,7.50,0.0,29.50,46.70,17.90,6.00
2,2001-09-03 00:00:00,26.50,36.60,18.20,49.30,71.30,24.10,23940.00,265.00,1.60,7.10,0.0,25.90,35.10,19.10,6.00
3,2001-09-04 00:00:00,24.90,32.40,19.70,62.30,84.90,39.90,24259.00,308.00,2.40,7.40,0.0,24.70,34.80,18.70,6.00
4,2001-09-05 00:00:00,22.30,30.40,17.00,68.30,93.80,32.80,25140.00,302.00,2.00,6.20,0.0,23.30,32.10,18.50,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7155,2021-03-25 23:58:59,13.42,22.26,5.78,69.86,96.10,32.79,23358.67,273.91,0.81,4.75,0.0,14.40,28.82,6.10,3.94
7156,2021-03-26 23:58:59,13.40,19.49,7.65,71.27,96.15,43.78,17075.67,355.38,1.19,4.66,0.0,14.47,25.44,8.37,2.80
7157,2021-03-27 23:58:59,16.13,24.13,9.27,57.01,80.30,27.95,20773.95,62.29,1.51,5.63,0.0,15.91,27.72,8.79,3.73
7158,2021-03-28 23:58:59,17.49,26.01,10.32,60.31,84.77,29.89,23048.76,127.27,1.68,5.16,0.0,17.85,31.80,10.11,4.34


In [3]:
# Set the index and clip only the precipitation data:
meteodata.set_index('Date', inplace=True)

# There are repetitive days in the table, thus we can exclude then:
meteodata=meteodata.resample('D').mean()

# We create a new column with the dates:
meteodata["dates"] = meteodata.index

In [4]:
meteodata.head()

,Tmed (ºC),Tmax (ºC),Tmin (ºC),HRmed (%),HRmax (%),HRmin (%),RSG (kj/m2),DV (graus),VVmed (m/s),VVmax (m/s),P (mm),Tmed Relva(ºC),Tmax Relva(ºC),Tmin Relva(ºC),ET0 (mm),dates
Date,,,,,,,,,,,,,,,,
2001-09-01,23.5,35.8,13.1,53.6,93.4,20.4,26200.0,225.0,1.6,6.1,0.0,28.6,47.6,16.2,6.0,2001-09-01
2001-09-02,25.1,33.7,16.7,48.5,69.8,27.7,25483.0,277.0,2.2,7.5,0.0,29.5,46.7,17.9,6.0,2001-09-02
2001-09-03,26.5,36.6,18.2,49.3,71.3,24.1,23940.0,265.0,1.6,7.1,0.0,25.9,35.1,19.1,6.0,2001-09-03
2001-09-04,24.9,32.4,19.7,62.3,84.9,39.9,24259.0,308.0,2.4,7.4,0.0,24.7,34.8,18.7,6.0,2001-09-04
2001-09-05,22.3,30.4,17.0,68.3,93.8,32.8,25140.0,302.0,2.0,6.2,0.0,23.3,32.1,18.5,6.0,2001-09-05


In [5]:
# We create a dataframe for the wet-dry and wet-wet:
pwetdry = pd.DataFrame(index = meteodata.index)

In [6]:
# We do the calculations:
pwetdry["wetdry"] = 0
pwetdry["wetwet"] = 0
yesterdayday = meteodata.index[0]
for todayday in meteodata.index[1:]:
    
    if (meteodata["P (mm)"].loc[todayday] > 0) & (meteodata["P (mm)"].loc[yesterdayday] == 0): 
        pwetdry["wetdry"].loc[todayday] = 1
    else:
        1+1
        
    if (meteodata["P (mm)"].loc[todayday] > 0) & (meteodata["P (mm)"].loc[yesterdayday] > 0): 
        pwetdry["wetwet"].loc[todayday] = 1
    else:
        1+1
        
    yesterdayday = todayday

In [7]:
# We resample the data monthly:
pwetdry_mon=pwetdry.resample('M').sum()
pwetdry_mon["month"] = pwetdry_mon.index
pwetdry_mon["month"]  = pwetdry_mon["month"].dt.month

In [8]:
pwetdry_mon

,wetdry,wetwet,month
Date,,,
2001-09-30,2,5,9
2001-10-31,3,15,10
2001-11-30,3,3,11
2001-12-31,8,5,12
2002-01-31,4,8,1
...,...,...,...
2020-11-30,6,9,11
2020-12-31,4,18,12
2021-01-31,3,19,1
